<a href="https://colab.research.google.com/github/IllgamhoDuck/googleXproject/blob/master/Quantum_simulator_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quantum Simulator Benchmark
**Investigate the speed of quantum simulator**
- numpy
- jax
- tensorflow
- IBM qiskit
- Google cirq


## Setting environment
- package install
- package import
- jupyter notebook configuration and color setting
- quantum simulator code

In [0]:
!pip install --quiet protobuf==3.8.0 networkx==2.3 tensornetwork jax jaxlib cirq qiskit nxpd pylatexenc jupyter_contrib_nbextensions

     |████████████████████████████████| 1.2MB 2.8MB/s 
     |████████████████████████████████| 1.8MB 48.1MB/s 
     |████████████████████████████████| 235kB 50.8MB/s 
     |████████████████████████████████| 1.2MB 46.5MB/s 
     |████████████████████████████████| 122kB 45.6MB/s 
     |████████████████████████████████| 20.9MB 155kB/s 
     |████████████████████████████████| 2.9MB 27.1MB/s 
     |████████████████████████████████| 5.3MB 35.0MB/s 
     |████████████████████████████████| 2.7MB 29.4MB/s 
     |████████████████████████████████| 18.2MB 167kB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
     |████████████████████████████████| 102kB 12.3MB/s 
     |████████████████████████████████| 1.4MB 32.5MB/s 
     |████████████████████████████████| 870kB 49.9MB/s 
     |████████████████████████████████| 481kB 39.5MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 51kB 7.5MB/s 
     |████████████████████████████████| 71kB 9.0MB/s 


In [0]:
%tensorflow_version 2.x
%matplotlib inline

# qiskit - test and visualize
from qiskit import QuantumCircuit
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import execute, Aer
from qiskit.providers.aer import QasmSimulator

# cirq - test and visualize
import cirq
from cirq import Circuit, Simulator
from cirq.devices import GridQubit, LineQubit
from cirq.contrib.qasm_import import circuit_from_qasm

# tensornetwork
import numpy as np
import jax
import tensornetwork as tn
from tensornetwork.backend_contextmanager import get_default_backend
# np = jax.numpy

# tensorflow
import tensorflow as tf

# math
import math

# test
import os
import time
import random
from tqdm import tqdm_notebook as tqdm

# benchmark
from pandas import Series, DataFrame
import pandas as pd
from google.colab import files
# !jupyter contrib nbextension install --user
# !jupyter nbextension enable codefolding/main

# multiproccessing
import multiprocessing as mp

TensorFlow 2.x selected.


In [0]:
qasm_simulator = Aer.get_backend('qasm_simulator')
statevector_simulator = Aer.get_backend('statevector_simulator')

class colors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    CEND      = '\33[0m'
    CBOLD     = '\33[1m'
    CITALIC   = '\33[3m'
    CURL      = '\33[4m'
    CBLINK    = '\33[5m'
    CBLINK2   = '\33[6m'
    CSELECTED = '\33[7m'

    CBLACK  = '\33[30m'
    CRED    = '\33[31m'
    CGREEN  = '\33[32m'
    CYELLOW = '\33[33m'
    CBLUE   = '\33[34m'
    CVIOLET = '\33[35m'
    CBEIGE  = '\33[36m'
    CWHITE  = '\33[37m'

    CBLACKBG  = '\33[40m'
    CREDBG    = '\33[41m'
    CGREENBG  = '\33[42m'
    CYELLOWBG = '\33[43m'
    CBLUEBG   = '\33[44m'
    CVIOLETBG = '\33[45m'
    CBEIGEBG  = '\33[46m'
    CWHITEBG  = '\33[47m'

    CGREY    = '\33[90m'
    CRED2    = '\33[91m'
    CGREEN2  = '\33[92m'
    CYELLOW2 = '\33[93m'
    CBLUE2   = '\33[94m'
    CVIOLET2 = '\33[95m'
    CBEIGE2  = '\33[96m'
    CWHITE2  = '\33[97m'

    CGREYBG    = '\33[100m'
    CREDBG2    = '\33[101m'
    CGREENBG2  = '\33[102m'
    CYELLOWBG2 = '\33[103m'
    CBLUEBG2   = '\33[104m'
    CVIOLETBG2 = '\33[105m'
    CBEIGEBG2  = '\33[106m'
    CWHITEBG2  = '\33[107m'

color = colors

In [0]:
class QuantumSimulator():
    """Quantum simulator with tensornetwork"""
    def __init__(self, qbit_n):
        """
        Args:
            qbit_n: The number of total qubit size
            circuit: Where to store gate
        """
        assert qbit_n > 0, "Qubit size should be at least 1"
        self.qbit_n = qbit_n
        self.circuit = []

        # tensorflow
        if tf.__version__.split('.')[0] == '1':
            self.sess = tf.Session()

    def initialize_circuit(self):
        """
        Initialize the circuit to execute

        Args:
            qbits: Store the first initialized qubits
            measures: Store the edge that used for measure for each qubit
        """
        # Initialize qubit
        self.qbits = [tn.Node(np.array([1 + 0j, 0 + 0j], dtype=np.complex64)) for _ in range(self.qbit_n)]

        # Storing the Edge node that will be used to measure for each qubit
        self.measures = [self.qbits[i][0] for i in range(self.qbit_n)]
        self.amplitude = None
        self.result = {}
        self.result_prob = {}
    
    def connect_qubits(self):
        """
        Make the seperate tensornetworks to one tensornetwork using
        2 qubit identity gate
        CI - Control Identity
             [[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]]
        """
        for i in range(self.qbit_n - 1):
            self.add_ci(i, i + 1)

    def x(self, qbit_i):
        """
        Add X gate to specific 1 qubit index 
        x(qbit_i=0)
        """
        self.circuit.append(('x', qbit_i))

    def y(self, qbit_i):
        """
        Add Y gate to specific 1 qubit index 
        y(qbit_i=5)
        """
        self.circuit.append(('y', qbit_i))

    def z(self, qbit_i):
        """
        Add Z gate to specific 1 qubit index 
        z(qbit_i=3)
        """
        self.circuit.append(('z', qbit_i))

    def h(self, qbit_i):
        """
        Add H gate to specific 1 qubit index 
        h(qbit_i=2)
        """
        self.circuit.append(('h', qbit_i))

    def t(self, qbit_i):
        """
        Add T gate to specific 1 qubit index 
        t(qbit_i=0)
        """
        self.circuit.append(('t', qbit_i))

    def cx(self, qbit_c, qbit_t):
        """
        Add CX gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        cx(qbit_c=0, qbit_t=1)
        """
        self.circuit.append(('cx', qbit_c, qbit_t))

    def cy(self, qbit_c, qbit_t):
        """
        Add CY gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        cy(qbit_c=3, qbit_t=2)
        """
        self.circuit.append(('cy', qbit_c, qbit_t))

    def cz(self, qbit_c, qbit_t):
        """
        Add CZ gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        cz(qbit_c=2, qbit_t=4)
        """
        self.circuit.append(('cz', qbit_c, qbit_t))

    def ch(self, qbit_c, qbit_t):
        """
        Add CH gate to specific 2 qubit(control / target) index
        control and target qubit must be different 
        ch(qbit_c=1, qbit_t=3)
        """
        self.circuit.append(('ch', qbit_c, qbit_t))
    
    def ccx(self, qbit_c1, qbit_c2, qbit_t):
        """
        Add CCX gate to specific 3 qubit(control1 / control2 / target) index
        control and target qubit must be different 
        ccx(qbit_c1=0, qbit_c2=1 qbit_t=2)
        """
        self.circuit.append(('ccx', qbit_c1, qbit_c2, qbit_t))

    def add_x(self, qbit_i):
        X = tn.Node(np.array([[0, 1],
                              [1, 0]], dtype=np.complex64))
        self.measures[qbit_i] ^ X[0]
        self.measures[qbit_i] = X[1]

    def add_y(self, qbit_i):
        Y = tn.Node(np.array([[0, 1j],
                              [-1j, 0]], dtype=np.complex64))
        self.measures[qbit_i] ^ Y[0]
        self.measures[qbit_i] = Y[1]

    def add_z(self, qbit_i):
        Z = tn.Node(np.array([[1, 0],
                              [0, -1]], dtype=np.complex64))
        self.measures[qbit_i] ^ Z[0]
        self.measures[qbit_i] = Z[1]

    def add_h(self, qbit_i):
        h_f = 1/math.sqrt(2)
        H = tn.Node(np.array([[h_f, h_f],
                              [h_f, -h_f]], dtype=np.complex64))
        self.measures[qbit_i] ^ H[0]
        self.measures[qbit_i] = H[1]

    def add_t(self, qbit_i):
        e_j_pi = math.e ** ((1j * math.pi) / 4)
        T = tn.Node(np.array([[1, 0],
                              [0, e_j_pi]], dtype=np.complex64))
        self.measures[qbit_i] ^ T[0]
        self.measures[qbit_i] = T[1]

    def add_cx(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        cx = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 0, 1],
                       [0, 0, 1, 0]], dtype=np.complex64)
        CX = tn.Node(cx.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CX[0]
        self.measures[qbit_t] ^ CX[1]
        self.measures[qbit_c] = CX[2]
        self.measures[qbit_t] = CX[3]

    def add_cy(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        cy = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 0, 1j],
                       [0, 0, -1j, 0]], dtype=np.complex64)
        CY = tn.Node(cy.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CY[0]
        self.measures[qbit_t] ^ CY[1]
        self.measures[qbit_c] = CY[2]
        self.measures[qbit_t] = CY[3]


    def add_cz(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        cz = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, -1]], dtype=np.complex64)
        CZ = tn.Node(cz.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CZ[0]
        self.measures[qbit_t] ^ CZ[1]
        self.measures[qbit_c] = CZ[2]
        self.measures[qbit_t] = CZ[3]

    def add_ch(self, qbit_c, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
            h_f: hadamard factor
        """
        h_f = 1/math.sqrt(2)
        ch = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, h_f, h_f],
                       [0, 0, h_f, -h_f]], dtype=np.complex64)
        CH = tn.Node(ch.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CH[0]
        self.measures[qbit_t] ^ CH[1]
        self.measures[qbit_c] = CH[2]
        self.measures[qbit_t] = CH[3]

    def add_ci(self, qbit_c, qbit_t):
        """
        Control Identity matrix for connect
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        ci = np.array([[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]], dtype=np.complex64)
        CI = tn.Node(ci.reshape((2, 2, 2, 2)))
        self.measures[qbit_c] ^ CI[0]
        self.measures[qbit_t] ^ CI[1]
        self.measures[qbit_c] = CI[2]
        self.measures[qbit_t] = CI[3]

    def add_ccx(self, qbit_c1, qbit_c2, qbit_t):
        """
        Args:
            qbit_c: Quantum qubit control
            qbit_t: Quantum qubit target
        """
        ccx = np.array([[1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 1, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 0, 0, 1, 0, 0],
                        [0, 0, 0, 0, 0, 0, 0, 1],
                        [0, 0, 0, 0, 0, 0, 1, 0]], dtype=np.complex64)
        CCX = tn.Node(ccx.reshape((2, 2, 2, 2, 2, 2)))
        self.measures[qbit_c1] ^ CCX[0]
        self.measures[qbit_c2] ^ CCX[1]
        self.measures[qbit_t] ^ CCX[2]
        self.measures[qbit_c1] = CCX[3]
        self.measures[qbit_c2] = CCX[4]
        self.measures[qbit_t] = CCX[5]

    def generate_qiskit_circuit(self):
        """Visualize the gate using Qiskit"""
        qasm = self.circuit_to_qasm()
        ibmq_circuit = QuantumCircuit.from_qasm_str(qasm)
        return ibmq_circuit

    def circuit_to_qasm(self):
        circuit = self.circuit
        qubit_size = self.qbit_n

        # Add meta data
        qasm_text = 'OPENQASM 2.0;\ninclude "qelib1.inc";\n'

        # Add quantum bits and classical bits
        qasm_text += 'qreg q[{0}];\ncreg c[{0}];\n'.format(qubit_size)

        # Add quantum gates
        for gate in circuit:
            # 1 qubit gate
            if len(gate) == 2:
                qasm_text += '{0} q[{1}];\n'.format(gate[0], gate[1])
            # 2 qubit gate
            elif len(gate) == 3:
                qasm_text += '{0} q[{1}],q[{2}];\n'.format(gate[0], gate[1], gate[2])
            elif len(gate) == 4:
                qasm_text += '{0} q[{1}],q[{2}],q[{3}];\n'.format(gate[0],
                                                                  gate[1],
                                                                  gate[2],
                                                                  gate[3])
            else:
                raise NotImplementedError("Quantum simulator doesn't support qubit gates more than 3")

        # Add measure gate
        for qbit_i in range(qubit_size):
            qasm_text += 'measure q[{0}] -> c[{0}];\n'.format(qbit_i)

        return qasm_text

    def qasm_to_circuit(self, qasm):
        self.circuit = []

        # Bool check
        FORMAT_CHECK = False
        LIB_CHECK = False
        QBIT_CHECK = False
        CBIT_CHECK = False

        # Regular Expression
        qbit_reg = re.compile('^q\[[0-9]+\];\\n$')
        cbit_reg = re.compile('^[a-zA-Z0-9_]+\[[0-9]+\];\\n$')
        num_reg = re.compile('[0-9]+')
        text_reg = re.compile('[a-zA-Z0-9_]+')
        gate1_reg = re.compile('^q\[[0-9]+\];\\n$')
        gate2_reg = re.compile('^q\[[0-9]+\],q\[[0-9]+\];\\n$')
        gate3_reg = re.compile('^q\[[0-9]+\],q\[[0-9]+\],q\[[0-9]+\];\\n$')
        meas_q_reg = re.compile('^q\[[0-9]+\]$')
        meas_c_reg = re.compile('^[a-zA-Z0-9_]+\[[0-9]+\];$')

        # Gate
        gate_1 = ['x', 'y', 'z', 'h', 't']
        gate_2 = ['cx', 'cy', 'cz', 'ch']
        gate_3 = ['ccx']

        # Analyze qasm one line by one line
        for one_line in io.StringIO(qasm):

            # Ignore comment and whitespace
            if one_line[:2] == '//' or one_line == '\n':
                continue

            # Check the format of the QASM
            if (FORMAT_CHECK == False or LIB_CHECK == False):
                if (FORMAT_CHECK == False and one_line == 'OPENQASM 2.0;\n'):
                    FORMAT_CHECK = True
                    continue
                elif (LIB_CHECK == False and one_line == 'include "qelib1.inc";\n'):
                    LIB_CHECK = True
                    continue
                elif FORMAT_CHECK == True:
                    raise TypeError("Only qelib1.inc lib is supported")
                elif LIB_CHECK == True:
                    raise TypeError("Only openqasm 2.0 format is supported")
                else:
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))

            # Check the quantum bit
            if (QBIT_CHECK == False):
                c_list = one_line.split(' ')
                if len(c_list) != 2:
                    print("qbit or cbit should be chosen")
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))
                if (QBIT_CHECK == False and c_list[0] == 'qreg'):
                    if (qbit_reg.match(c_list[1])):
                        self.qbit_n = int(num_reg.search(c_list[1]).group())
                        QBIT_CHECK = True
                        continue
                else:
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))
            
            c_list = one_line.split(' ')
            # Check the classic bit and this is optional
            if (CBIT_CHECK == False and c_list[0] == 'creg'):
                if (cbit_reg.match(c_list[1])):
                    # Check the cbit information
                    # TODO: Not used currently!
                    # text_reg.search(c_list[1]).group()
                    # int(num_reg.search(c_list[1]).group())
                    CBIT_CHECK = True
                    continue
                else:
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))
            else:
                CBIT_CHECK = True
            
            # Gate check
            c_list = one_line.split(' ')
            gate = c_list[0]
            if (gate in gate_1):
                if len(c_list) != 2:
                    print("qbit or cbit should be chosen")
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))
                if gate1_reg.match(c_list[1]):
                    self.circuit.append((gate, int(num_reg.search(c_list[1]).group())))
                    continue
                else:
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))
            elif (gate in gate_2):
                if len(c_list) != 2:
                    print("qbit or cbit should be chosen")
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))
                if gate2_reg.match(c_list[1]):
                    q_list = tuple(num_reg.findall(c_list[1]))
                    self.circuit.append((gate,) + q_list)
                    continue
                continue
            elif (gate in gate_3):
                if len(c_list) != 2:
                    print("qbit or cbit should be chosen")
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))
                if gate3_reg.match(c_list[1]):
                    q_list = tuple(num_reg.findall(c_list[1]))
                    self.circuit.append((gate,) + q_list)
                    continue
                continue
            elif (gate == 'measure'):
                if len(c_list) != 4 or c_list[2] != '->':
                    print("qbit or cbit should be chosen")
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))
                if (meas_q_reg.match(c_list[1]) and meas_c_reg.match(c_list[3])):
                    # Check the cbit information
                    # TODO: Not used currently!
                    int(num_reg.search(c_list[1]).group())
                    text_reg.search(c_list[3]).group()
                    int(num_reg.search(c_list[3]).group())
                else:
                    raise TypeError("Wrong syntax of QASM: {}".format(one_line))
                continue
            else:
                raise NotImplementedError("Gate {} is not supported".format(gate))

    def generate_result(self):
        for i in range(2 ** self.qbit_n):
            # Generate statevector
            statevector = ""
            for shift in range(self.qbit_n):
                statevector = str((i >> shift) & 1) + statevector
            
            # Check the amplitude
            # tensorflow and other backend is calculated differently
            amp = self.amplitude[self.bitstring_to_int(statevector)]
            if get_default_backend() == 'tensorflow' and tf.__version__.split('.')[0] == '1':
                pass
            elif get_default_backend() == 'tensorflow':
                amp = amp.numpy()
            else:
                amp = complex(str(amp))

            if amp == 0:
                continue

            # TODO: Figure out how to get value out of jax array
            self.result[statevector] = amp 
            self.result_prob[statevector] = (amp*amp.conjugate()).real

    def bitstring_to_int(self, bitstring):
        int_list = []
        for i in bitstring[::-1]:
            int_list.append(int(i))
        return tuple(int_list)

    def execute_circuit(self):
        """
        gate is structed as following
        1 qubit gate - (gate_name, qubit_index)
        2 qubit gate - (gate_name, qubit_control_index, qubit_target_index)
        """
        gate_1 = ['x', 'y', 'z', 'h', 't']
        gate_2 = ['cx', 'cy', 'cz', 'ch']
        gate_3 = ['ccx']
        gate_1_func = [self.add_x, self.add_y, self.add_z, self.add_h, self.add_t]
        gate_2_func = [self.add_cx, self.add_cy, self.add_cz, self.add_ch]
        gate_3_func = [self.add_ccx]

        # Connect the qubits
        for gate in self.circuit:
            if gate[0] in gate_2:
                i = gate_2.index(gate[0])
                gate_2_func[i](gate[1], gate[2])
            elif gate[0] in gate_1:
                i = gate_1.index(gate[0])
                gate_1_func[i](gate[1])
            elif gate[0] in gate_3:
                i = gate_3.index(gate[0])
                gate_3_func[i](gate[1], gate[2], gate[3])
            else:
                raise NotImplementedError("Gate {} is not supported".format(gate[0]))

        nodes = tn.reachable(self.qbits[0])
        self.amplitude = tn.contractors.greedy(nodes, output_edge_order=self.measures).tensor

        # Tensorflow session run only at tensorflow 1
        if get_default_backend() == 'tensorflow' and tf.__version__.split('.')[0] == '1':
            self.amplitude = self.sess.run(self.amplitude)

    def clear_circuit(self):
        self.circuit = []

    def change_qubit_size(self, size):
        assert size > 0, "Qubit size should be at least 1"
        self.qbit_n = size

    def execute(self):
        assert self.qbit_n > 0, "Qubit size should be at least 1"
        self.initialize_circuit()
        self.connect_qubits()
        self.execute_circuit()


## Quantum Simulator Test

#### Quantum simulator test code

In [0]:
class QuantumSimulatorTest():
    """Test the quantum simulator"""
    def __init__(self,
                 min_qubit_size=1,
                 max_qubit_size=5,
                 min_gate_size=1,
                 max_gate_size=40):
        """
        Choose the testing enviroment
        Args:
            min_qubit_size: minimum number of the qubit size
            min_qubit_size: maximum number of the qubit size
            min_qubit_size: minimum number of the gate size
            min_qubit_size: maximum number of the gate size
        """
        self.max_qubit_size = max_qubit_size
        self.min_qubit_size = min_qubit_size
        self.max_gate_size = max_gate_size
        self.min_gate_size = min_gate_size

    def __call__(self,
                 quantum_simulator,
                 ibmq_backend,
                 cirq_backend,
                 test_size=10,
                 verbose=True,
                 debug=False):
        """
        Process test
        Quantum simulator testing using ibmq qiskit
        Args:
            quantum_simulator: quantum simulator that will be used to simulate
            ibmq_backend: ibmq simulator that will be used as reference
                - qasm_simulator: show the number of shots for each statevector
                - statevector_simualtor: show the amplitude and prob for each statevector
                                         measure gates are disabled
            test_size: total test case size
            verbose: Print the value
        """
        # Check IBMQ backend
        if ibmq_backend:
            self.ibmq_backend = ibmq_backend.configuration().backend_name
        else:
            self.ibmq_backend = None

        # Get cirq backend type
        if cirq_backend == None:
            self.cirq_backend = None
        elif cirq_backend == 'count' or cirq_backend == 'density':
            self.cirq_backend = cirq_backend
        else:
            raise NotImplementedError('Only Count or Density backend is supported')
        self.cirq_simulator = cirq.Simulator()

        # Init variable
        ibmq_end_time = 0
        ibmq_start_time = 0
        cirq_end_time = 0
        cirq_start_time = 0

        # Check the correct
        total = 0
        correct = 0
        test_start_time = time.time()

        # tqdm configuration
        tqdm_disable = False if verbose else True

        # Run the test!
        for test_i in tqdm(range(test_size), disable=tqdm_disable):
            # Generate the random circuit and qasm(IBM Qiskit)
            random_qubit_size = random.randint(self.min_qubit_size, self.max_qubit_size)
            random_gate_size = random.randint(self.min_gate_size, self.max_gate_size)
            circuit, ibmq_qasm, cirq_qasm = self.random_circuit(random_qubit_size, random_gate_size)

            # Print out the current test case information
            print(color.BOLD)
            print('TEST CASE [{} / {}] qubit: {} gate: {}'.format(test_i + 1,
                                                                test_size,
                                                                random_qubit_size,
                                                                random_gate_size))
            print(color.ENDC)

            # Run the Quantum simulator and get the result
            qs_start_time = time.time()
            quantum_simulator.change_qubit_size(random_qubit_size)
            quantum_simulator.circuit = circuit
            quantum_simulator.execute()
            qs_end_time = time.time()

            self.qs_result = quantum_simulator.result_prob

            # Run IBM qiskit simulator and get the result
            if self.ibmq_backend:
                shots = 1000 # default

                ibmq_start_time = time.time()
                ibmq_circuit = QuantumCircuit.from_qasm_str(ibmq_qasm)
                self.ibmq_result = execute(ibmq_circuit, ibmq_backend, shots=shots).result()
                ibmq_end_time = time.time()

            # Run Google cirq simulator and get the result
            if self.cirq_backend:
                shots = 1000 # default

                cirq_start_time = time.time()
                cirq_circuit = circuit_from_qasm(cirq_qasm)
                if self.cirq_backend == 'count':
                    self.cirq_result = self.cirq_simulator.run(cirq_circuit, repetitions=shots)
                elif self.cirq_backend == 'density':
                    self.cirq_result = self.cirq_simulator.simulate(cirq_circuit)
                cirq_end_time = time.time()

            ############################################################
            ### PRINT OUT THE INFORMATIONS #############################
            ############################################################

            if verbose:
                # 1. Circuit information
                print(circuit)
                if self.ibmq_backend:
                    print(ibmq_circuit)

                # 2. Quantum simulation
                quantum_simulator.generate_result()
                print(color.BOLD + '[ QUANTUM SIMULATOR ]' + color.ENDC)
                print(color.CURL, end="")
                print("Quantum simulator tested time: %s sec\n" % format(qs_end_time - qs_start_time))
                print(color.ENDC, end="")
                print(quantum_simulator.result)
                print(self.qs_result)
                print('')

                # 3. IBMQ simulation
                if self.ibmq_backend:
                    print(color.BOLD + '[ IBM QISKIT ]' + color.ENDC)
                    print(color.CURL, end="")
                    print("IBMQ tested time: %s sec\n" % format(ibmq_end_time - ibmq_start_time))
                    print(color.ENDC, end="")
                    if self.ibmq_backend == 'qasm_simulator':
                        print(self.ibmq_result.get_counts())
                    if self.ibmq_backend == 'statevector_simulator':
                        self.generate_ibmq_result(random_qubit_size)
                        print(self.ibmq_amp)
                        print(self.ibmq_prob)
                    print('')                    

                # 4. Cirq simulation
                if self.cirq_backend:
                    print(color.BOLD + '[ GOOGLE CIRQ ]' + color.ENDC)
                    print(color.CURL, end="")

                    print("Cirq tested time: %s sec\n" % format(cirq_end_time - cirq_start_time))
                    print(color.ENDC, end="")
                    if self.cirq_backend == 'count':
                        self.preprocess_cirq_count(random_qubit_size)
                        print(self.cirq_count)
                    if self.cirq_backend == 'density':
                        self.generate_cirq_result(random_qubit_size)
                        print(self.cirq_amp)
                        print(self.cirq_prob)
                    print('')

                # 5. Compare the result
                if self.ibmq_backend or self.cirq_backend:
                    print(color.BOLD + '[ COMPARE ]' + color.ENDC)

                ## IBMQ
                if self.ibmq_backend:
                    cmp_start_time = time.time()
                    error_n_ibmq, cmp_total_ibmq, total_error_ibmq = self.compare_probability_ibmq(shots)
                    print(color.OKBLUE + '[ IBM QISKIT ]' + color.ENDC, end=" ")
                    print(color.CURL, end="")
                    print("IBMQ Comparing time: %s sec" % format(time.time() - cmp_start_time))
                    print(color.ENDC)
                    if error_n_ibmq == 0:
                        print(color.OKBLUE + 'ERROR OCCUR / TOTAL CHECK: ' + color.ENDC, error_n_ibmq, '/', cmp_total_ibmq)
                        print(color.OKBLUE + 'TOTAL ERROR: ' + color.ENDC, total_error_ibmq,'\n')
                    else:
                        print(color.FAIL + 'ERROR OCCUR / TOTAL CHECK: ' + color.ENDC, error_n_ibmq, '/', cmp_total_ibmq)
                        print(color.FAIL + 'TOTAL ERROR: ' + color.ENDC, total_error_ibmq,'\n')

                ## CIRQ
                if self.cirq_backend:
                    cmp_start_time = time.time()
                    error_n_cirq, cmp_total_cirq, total_error_cirq = self.compare_probability_cirq(shots)
                    print(color.OKBLUE + '[ GOOGLE CIRQ ]' + color.ENDC, end=" ")
                    print(color.CURL, end="")
                    print("CIRQ Comparing time: %s sec" % format(time.time() - cmp_start_time))
                    print(color.ENDC)
                    if error_n_cirq == 0:
                        print(color.OKBLUE + 'ERROR OCCUR / TOTAL CHECK: ' + color.ENDC, error_n_cirq, '/', cmp_total_cirq)
                        print(color.OKBLUE + 'TOTAL ERROR: ' + color.ENDC, total_error_cirq,'\n')
                    else:
                        print(color.FAIL + 'ERROR OCCUR / TOTAL CHECK: ' + color.ENDC, error_n_cirq, '/', cmp_total_cirq)
                        print(color.FAIL + 'TOTAL ERROR: ' + color.ENDC, total_error_cirq,'\n')
                
                if self.ibmq_backend and self.cirq_backend:
                    if error_n_ibmq == 0 and error_n_cirq == 0:
                        correct += 1
                elif self.ibmq_backend:
                    if error_n_ibmq == 0:
                        correct += 1
                elif self.cirq_backend:
                    if error_n_cirq == 0:
                        correct += 1
                if self.ibmq_backend or self.cirq_backend:
                    total += 1
            else:
                print("Quantum simulator tested time: %s sec" % format(qs_end_time - qs_start_time))
                if self.ibmq_backend:
                    print("IBMQ tested time: %s sec" % format(ibmq_end_time - ibmq_start_time))
                if self.cirq_backend:
                    print("Cirq tested time: %s sec\n" % format(cirq_end_time - cirq_start_time))

            ############################################################
            ############################################################
            ############################################################

        # Print out the final result
        if verbose:
            if self.ibmq_backend or self.cirq_backend:
                if correct == total:
                    print(color.OKBLUE + "The final result is {} / {}".format(correct, total) + color.ENDC)
                else:
                    print(color.FAIL + "The final result is {} / {}".format(correct, total) + color.ENDC)
                print("Total tested time: %s sec" % format(time.time() - test_start_time))

        if debug:
            return (random_qubit_size,
                    random_gate_size,
                    qs_end_time - qs_start_time,
                    ibmq_end_time - ibmq_start_time,
                    cirq_end_time - cirq_start_time)

    def generate_ibmq_result(self, qubit_size):
        """
        Generate statevector dictionary stores probability of ibmq simulator
        based on amplitude from ibmq statevector simulator
        """
        ibmq_amplitude = self.ibmq_result.data()['statevector']
        self.ibmq_amp = {}
        self.ibmq_prob = {}
        for i in range(2 ** qubit_size):
            # Generate statevector
            statevector = ""
            for shift in range(qubit_size):
                statevector = str((i >> shift) & 1) + statevector

            # Check the amplitude
            amp_list = ibmq_amplitude[i]
            amp = complex(amp_list[0], amp_list[1])

            if amp == 0:
                continue

            self.ibmq_amp[statevector] = amp
            self.ibmq_prob[statevector] = (amp*amp.conjugate()).real
    
    def generate_cirq_result(self, qubit_size):
        """
        Generate statevector dictionary stores probability of cirq simulator
        based on amplitude from cirq statevector simulator
        """
        cirq_amplitude = self.cirq_result.final_state
        self.cirq_amp = {}
        self.cirq_prob = {}
        for i in range(2 ** qubit_size):
            # Generate statevector
            statevector = ""
            for shift in range(qubit_size):
                statevector = str((i >> shift) & 1) + statevector
            statevector = statevector[::-1]

            # Check the amplitude
            amp = cirq_amplitude[i]

            if amp == 0:
                continue

            self.cirq_amp[statevector] = amp
            self.cirq_prob[statevector] = (amp*amp.conjugate()).real

    def compare_probability_ibmq(self, shots, delta=1e-6):
        """
        Check each statevector probability and if the value is
        more than give delta(default: 1e-6) value consider as error
        """
        total_error = 0
        error_n = 0
        cmp_total = 0

        # Get statevector info from both simulator
        if self.ibmq_backend == 'qasm_simulator':
            ibmq_count = self.ibmq_result.get_counts()
            state_vectors = list(set(self.qs_result.keys()).union(set(ibmq_count.keys())))
        if self.ibmq_backend == 'statevector_simulator':
            state_vectors = list(set(self.qs_result.keys()).union(set(self.ibmq_amp.keys())))

        # Check is there is an error
        for state_vector in state_vectors:
            qs_val = self.qs_result.get(state_vector, 0)

            if self.ibmq_backend == 'qasm_simulator':
                ibmq_val = ibmq_count.get(state_vector, 0)/shots
            if self.ibmq_backend == 'statevector_simulator':
                ibmq_val = self.ibmq_prob.get(state_vector, 0)

            error = abs(qs_val - ibmq_val)
            total_error += error
            if error > delta:
                error_n += 1
            cmp_total += 1

        return error_n, cmp_total, total_error
    
    def compare_probability_cirq(self, shots, delta=1e-6):
        """
        Check each statevector probability and if the value is
        more than give delta(default: 1e-6) value consider as error
        """
        total_error = 0
        error_n = 0
        cmp_total = 0

        # Get statevector info from both simulator
        if self.cirq_backend == 'count':
            cirq_count = self.cirq_count
            state_vectors = list(set(self.qs_result.keys()).union(set(cirq_count.keys())))
        if self.cirq_backend == 'density':
            state_vectors = list(set(self.qs_result.keys()).union(set(self.cirq_amp.keys())))

        # Check is there is an error
        for state_vector in state_vectors:
            qs_val = self.qs_result.get(state_vector, 0)

            if self.cirq_backend == 'count':
                cirq_val = cirq_count.get(state_vector, 0)/shots
            if self.cirq_backend == 'density':
                cirq_val = self.cirq_prob.get(state_vector, 0)

            error = abs(qs_val - cirq_val)
            total_error += error
            if error > delta:
                error_n += 1
            cmp_total += 1

        return error_n, cmp_total, total_error
    
    def preprocess_cirq_count(self, qubit_size):
        self.cirq_count = {}
        keys = []
        for i in range(qubit_size):
            keys.append('m_{}'.format(i))
        cirq_count = self.cirq_result.multi_measurement_histogram(keys=keys)
        for statevector, count in cirq_count.items():
             # Generate statevector
            bitstring = ""
            for bit in statevector:
                bitstring = str(bit) + bitstring
            self.cirq_count[bitstring] = count

    def random_circuit(self, qubit_size=5, gate_size=20):
        """
        Args:
            gate_1: list of 1 qubit gates
            gate_2: list of 2 qubit gates
            qubit_list: list of qubit index
        """
        # Quantum Simulator
        gate_1 = ['x', 'y', 'z', 'h', 't']
        gate_2 = ['cx', 'cy', 'cz', 'ch']
        qubit_list = list(range(qubit_size))

        circuit = []
        for _ in range(gate_size):
            if qubit_size == 1 or random.random() > 0.5:
                circuit.append(tuple([random.choice(gate_1), *random.sample(qubit_list, 1)]))
            else:
                circuit.append(tuple([random.choice(gate_2), *random.sample(qubit_list, 2)]))

        # IBM Qiskit
        if self.ibmq_backend:
            ibmq_qasm = self.circuit_to_qasm(circuit, qubit_size, backend='ibmq')
        else:
            ibmq_qasm = None
        if self.cirq_backend:
            cirq_qasm = self.circuit_to_qasm(circuit, qubit_size, backend='cirq')
        else:
            cirq_qasm = None

        return circuit, ibmq_qasm, cirq_qasm

    def circuit_to_qasm(self, circuit, qubit_size, backend):
        # Check backend
        if backend != 'ibmq' and backend != 'cirq':
            raise NotImplementedError("{} backend is not supported".format(backend))
        
        # Add meta data
        qasm_text = 'OPENQASM 2.0;\ninclude "qelib1.inc";\n'
        
        # Add quantum bits and classical bits
        if backend == 'ibmq':
            qasm_text += 'qreg q[{0}];\ncreg c[{0}];\n'.format(qubit_size)
        elif backend == 'cirq':
            qasm_text += 'qreg q[{0}];\n'.format(qubit_size)
            if self.cirq_backend == 'count':
                qasm_text += 'creg m[{0}];\n'.format(qubit_size)

        # Add Identity gate to all qubit to prevent qubit dissapearing
        # When converting qasm to ciruit
        if backend == 'cirq':
            for i in range(qubit_size):
                qasm_text += 'id q[{0}];\n'.format(i)

        # Add quantum gates
        for gate in circuit:
            # 1 qubit gate
            if len(gate) == 2:
                qasm_text += '{0} q[{1}];\n'.format(gate[0], gate[1])
            # 2 qubit gate
            elif len(gate) == 3:
                qasm_text += '{0} q[{1}],q[{2}];\n'.format(gate[0], gate[1], gate[2])
            else:
                raise NotImplementedError("Quantum simulator doesn't support qubit gates more than 2")

        # Add measure gate
        if backend == 'ibmq' and self.ibmq_backend == 'qasm_simulator':
            for qbit_i in range(qubit_size):
                qasm_text += 'measure q[{0}] -> c[{0}];\n'.format(qbit_i)
        elif backend == 'cirq' and self.cirq_backend == 'count':
            for qbit_i in range(qubit_size):
                qasm_text += 'measure q[{0}] -> m[{0}];\n'.format(qbit_i)

        return qasm_text


#### **Quantum Simulator Test**

1. Choose the random simulator qubit & gate range


    min_qubit_size - Size of minumun qubit size to test
    max_qubit_size - Size of maximum qubit size to test
    min_gate_size - Size of minumun gate size to test
    max_gate_size - Size of maximum gate size to test

2. Test the simulator


    quantum_simulator - Built quantum simulator to be tested
    ibmq_backend - The simualtor will be used as backend
        - qasm_simulator: statevector count infomation (default 1000)
        - statevector_simulator: statevector amplitude & probability information
    test_size - Choose how many cases you want to test
    error_path - Choose the path where error case file will be saved
    error_file - Choose the error case file name 

In [0]:
t = QuantumSimulatorTest(min_qubit_size=24,
                         max_qubit_size=24,
                         min_gate_size=30,
                         max_gate_size=30)
circuit = QuantumSimulator(1)

In [0]:
t(quantum_simulator=circuit,
  ibmq_backend=statevector_simulator,
  cirq_backend='density',
  test_size=1,
  verbose=True)


TEST CASE [1 / 1] qubit: 10 gate: 6

[('x', 9), ('cy', 4, 6), ('x', 2), ('y', 6), ('x', 2), ('cy', 7, 6)]
                       
q_0: |0>───────────────
                       
q_1: |0>───────────────
        ┌───┐┌───┐     
q_2: |0>┤ X ├┤ X ├─────
        └───┘└───┘     
q_3: |0>───────────────
                       
q_4: |0>──■────────────
          │            
q_5: |0>──┼────────────
        ┌─┴─┐┌───┐┌───┐
q_6: |0>┤ Y ├┤ Y ├┤ Y ├
        └───┘└───┘└─┬─┘
q_7: |0>────────────■──
                       
q_8: |0>───────────────
        ┌───┐          
q_9: |0>┤ X ├──────────
        └───┘          
 c_0: 0 ═══════════════
                       
 c_1: 0 ═══════════════
                       
 c_2: 0 ═══════════════
                       
 c_3: 0 ═══════════════
                       
 c_4: 0 ═══════════════
                       
 c_5: 0 ═══════════════
                       
 c_6: 0 ═══════════════
                       
 c_7: 0 ═══════════════
                       
 c_8:

In [0]:
t(quantum_simulator=circuit,
  ibmq_backend=None,
  cirq_backend='density',
  test_size=10,
  verbose=False)

Quantum simulator tested time: 0.12169551849365234 sec
Cirq tested time: 2.381338596343994 sec



In [0]:
t(quantum_simulator=circuit,
  ibmq_backend=statevector_simulator,
  cirq_backend=None,
  test_size=1,
  verbose=True)

In [0]:
t(quantum_simulator=circuit,
  ibmq_backend=None,
  cirq_backend=None,
  test_size=1,
  verbose=True)


TEST CASE [1 / 1] qubit: 3 gate: 9

[('z', 1), ('y', 1), ('z', 0), ('cy', 0, 1), ('cy', 1, 0), ('t', 2), ('cy', 2, 1), ('t', 2), ('ch', 1, 0)]
[ QUANTUM SIMULATOR ]
Quantum simulator tested time: 0.08648157119750977 sec

{'010': (-0.70710677+0j), '011': (0.70710677+0j)}
{'010': 0.49999997, '011': 0.49999997}




## Quantum Simulator Benchmark

#### Quantum simulator benchmark code

In [0]:
class QuantumSimulatorBenchmark():
    """Benchmark of quantum simulator"""
    def __init__(self,
                 min_qubit_size=1,
                 max_qubit_size=20,
                 min_gate_size=1,
                 max_gate_size=100):
        """
        Choose the testing enviroment
        Args:
            min_qubit_size: minimum number of the qubit size to check
            min_qubit_size: maximum number of the qubit size to check
            min_qubit_size: minimum number of the gate size to check
            min_qubit_size: maximum number of the gate size to check
        """
        self.max_qubit_size = max_qubit_size
        self.min_qubit_size = min_qubit_size
        self.max_gate_size = max_gate_size
        self.min_gate_size = min_gate_size

    def check_cpu(self):
        print(color.BOLD + color.HEADER + '[ CPU ]' + color.ENDC)
        print("processor number: ", mp.cpu_count())
        cpu_infos = !cat /proc/cpuinfo
        for cpu_info in cpu_infos:
            if cpu_info.startswith('processor'):
                print("processor :", cpu_info.split(': ')[1], end=" ")
            if cpu_info.startswith('model name'):
                print(cpu_info.split(': ')[1])
        print('')

    def check_memory(self):
        # memory
        mem_infos = !cat /proc/meminfo
        for mem_info in mem_infos:
            if mem_info.startswith('MemTotal'):
                print("memory total :", mem_info.split(': ')[1])
            if mem_info.startswith('MemFree'):
                print("memory free :", mem_info.split(': ')[1])
            if mem_info.startswith('MemAvailable'):
                print("memory available :", mem_info.split(': ')[1])
        print('')
    
    def check_gpu(self):
        print(color.BOLD + color.HEADER + '[ GPU ]' + color.ENDC)
        !nvidia-smi
    
    def check_tpu(self):
        print(color.BOLD + color.HEADER + '[ TPU ]' + color.ENDC)
        try:
            tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
            print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
        except:
            print("No TPU found")
        print('')

    def check_jax(self):
        print(color.BOLD + '[ JAX BACKEND ]' + color.ENDC)
        print(jax.local_devices())
        print('')
    
    def check_tensorflow(self):
        print(color.BOLD + '[ TENSORFLOW BACKEND ]' + color.ENDC)
        print("tensorflow version: ", tf.__version__)
        gpu_device = tf.test.gpu_device_name()
        if gpu_device == '':
            print("device information: No gpu available")
        else:
            print("device information: ", gpu_device)
        print('')

    def check_device(self):
        # Check hardware spec
        self.check_cpu()
        self.check_memory()
        self.check_gpu()
        self.check_tpu()

        # Check backend
        self.check_jax()
        self.check_tensorflow()

    def run_benchmark(self):
        # Colab device information
        self.check_device()

        # Pandas data
        benchmark_data = {'qubit': [],
                          'gate': [],
                          'numpy': [],
                          'jax': [],
                          'tensorflow': [],
                          'qiskit': [],
                          'cirq': []}

        # Run benchmark test
        for qubit in range(self.min_qubit_size, self.max_qubit_size + 1):
            for gate in range(self.min_gate_size, self.max_gate_size + 1):

                # pandas qubit / gate
                benchmark_data['qubit'].append(qubit)
                benchmark_data['gate'].append(gate)
                
                result = []

                # NUMPY
                tn.set_default_backend('numpy')
                t = QuantumSimulatorTest(qubit, qubit, gate, gate)
                benchmark = t(circuit, None, None, 1, False, True)
                result.append(benchmark[2])

                # JAX
                tn.set_default_backend('jax')
                t = QuantumSimulatorTest(qubit, qubit, gate, gate)
                benchmark = t(circuit, None, None, 1, False, True)
                result.append(benchmark[2])

                # TENSORFLOW
                tn.set_default_backend('tensorflow')
                t = QuantumSimulatorTest(qubit, qubit, gate, gate)
                benchmark = t(circuit, statevector_simulator, 'density', 1, False, True)
                result.extend(benchmark[2:])

                # pandas time benchmark
                benchmark_data['numpy'].append(result[0])
                benchmark_data['jax'].append(result[1])
                benchmark_data['tensorflow'].append(result[2])
                benchmark_data['qiskit'].append(result[3])
                benchmark_data['cirq'].append(result[4])

                # Check the max & min index
                benchmark = np.array(result)
                max_i = np.argmax(benchmark)
                min_i = np.argmin(benchmark)

                print(color.BOLD + '[ QUBIT ]' + color.ENDC + ' ', '%2s' % str(qubit), end=" ")
                print(color.BOLD + '[ GATE ]' + color.ENDC + ' ', '%2s' % str(gate), end=" ")

                # Quantum Simulator - numpy
                if (min_i == 0):
                    print(color.BOLD + color.OKBLUE + '[ QS ', end='')
                    print(color.HEADER + color.BOLD + '<numpy>' + color.ENDC, end='')
                    print(color.BOLD + color.OKBLUE + ' ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.OKBLUE + '%-11s' % str(round(benchmark[0], 7)) + color.ENDC, end=" ")
                elif (max_i == 0):
                    print(color.BOLD + color.FAIL + '[ QS ', end='')
                    print(color.HEADER + color.BOLD + '<numpy>' + color.ENDC, end='')
                    print(color.BOLD + color.FAIL + ' ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.FAIL + '%-11s' % str(round(benchmark[0], 7)) + color.ENDC, end=" ")
                else:
                    print('[ QS <numpy> ] ', end='')
                    print('%-11s' % str(round(benchmark[0], 7)), end=" ")

                # Quantum Simulator - jax
                if (min_i == 1):
                    print(color.BOLD + color.OKBLUE + '[ QS ', end='')
                    print(color.HEADER + color.BOLD + '<jax>' + color.ENDC, end='')
                    print(color.BOLD + color.OKBLUE + ' ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.OKBLUE + '%-11s' % str(round(benchmark[1], 7)) + color.ENDC, end=" ")
                elif (max_i == 1):
                    print(color.BOLD + color.FAIL + '[ QS ', end='')
                    print(color.HEADER + color.BOLD + '<jax>' + color.ENDC, end='')
                    print(color.BOLD + color.FAIL + ' ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.FAIL + '%-11s' % str(round(benchmark[1], 7)) + color.ENDC, end=" ")
                else:
                    print('[ QS <jax> ] ', end='')
                    print('%-11s' % str(round(benchmark[1], 7)), end=" ")

                # Quantum Simulator - tensorflow
                if (min_i == 2):
                    print(color.BOLD + color.OKBLUE + '[ QS ', end='')
                    print(color.HEADER + color.BOLD + '<tensorflow>' + color.ENDC, end='')
                    print(color.BOLD + color.OKBLUE + ' ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.OKBLUE + '%-11s' % str(round(benchmark[2], 7)) + color.ENDC, end=" ")
                elif (max_i == 2):
                    print(color.BOLD + color.FAIL + '[ QS ', end='')
                    print(color.HEADER + color.BOLD + '<tensorflow>' + color.ENDC, end='')
                    print(color.BOLD + color.FAIL + ' ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.FAIL + '%-11s' % str(round(benchmark[2], 7)) + color.ENDC, end=" ")
                else:
                    print('[ QS <tensorflow> ] ', end='')
                    print('%-11s' % str(round(benchmark[2], 7)), end=" ")

                # IBMQ - QISKIT
                if (min_i == 3):
                    print(color.BOLD + color.OKBLUE + '[ QISKIT ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.OKBLUE + '%-11s' % str(round(benchmark[3], 7)) + color.ENDC, end=" ")
                elif (max_i == 3):
                    print(color.BOLD + color.FAIL + '[ QISKIT ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.FAIL + '%-11s' % str(round(benchmark[3], 7)) + color.ENDC, end=" ")
                else:
                    print('[ QISKIT ] ', end='')
                    print('%-11s' % str(round(benchmark[3], 7)), end=" ")

                # GOOGLE - CIRQ
                if (min_i == 4):
                    print(color.BOLD + color.OKBLUE + '[ CIRQ ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.OKBLUE + '%-11s' % str(round(benchmark[4], 7)) + color.ENDC)
                elif (max_i == 4):
                    print(color.BOLD + color.FAIL + '[ CIRQ ]' + color.ENDC + ' ', end='')
                    print(color.BOLD + color.FAIL + '%-11s' % str(round(benchmark[4], 7)) + color.ENDC)
                else:
                    print('[ CIRQ ] ', end='')
                    print('%-11s' % str(round(benchmark[4], 7)))
        print('')

        # Pandas DataFrame
        benchmark_pd = pd.DataFrame(benchmark_data, columns=['qubit', 'gate', 'numpy', 'jax', 'tensorflow', 'qiskit', 'cirq'])
        return benchmark_pd


#### **Quantum Simulator Benchmark**

1. Choose the Benchmark qubit & gate range


    min_qubit_size - Size of minumun qubit size to test
    max_qubit_size - Size of maximum qubit size to test
    min_gate_size - Size of minumun gate size to test
    max_gate_size - Size of maximum gate size to test

2. Test the simulator


    Just run :)

In [0]:
circuit = QuantumSimulator(1)

In [0]:
# multi processing test
def count(test):
    for i in range(1, 10000):
        pass

# input
test = [0 for _ in range(10000)]

# single processor
start_time = time.time()
for i in test:
    count(i)
print(round(time.time() - start_time, 5), "sec")

# multi processor
pool = mp.Pool(processes=mp.cpu_count())
start_time = time.time()
pool.map(count, test)
print(round(time.time() - start_time, 5), "sec")

1.99925 sec
1.18547 sec


#### **CPU**

##### **Fixed Gate Size 30**

In [0]:
# Fixed Gate Size 30
cpu = QuantumSimulatorBenchmark(min_qubit_size=1,
                              max_qubit_size=23,
                              min_gate_size=30,
                              max_gate_size=30)
cpu_benchmark_1 = cpu.run_benchmark()

[ CPU ]
processor : 0 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 1 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 2 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 3 Intel(R) Xeon(R) CPU @ 2.30GHz

[ GPU ]
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

[ TPU ]
No TPU found

[ JAX BACKEND ]
[CpuDevice(id=0)]

[ TENSORFLOW BACKEND ]
tensorflow version:  2.1.0
device information: No gpu available

[ QUBIT ]   1 [ GATE ]  30 [ QS <numpy> ] 0.0058448   [ QS <jax> ] 0.0322087   [ QS <tensorflow> ] 0.0170436   [ QISKIT ] 0.0623226   [ CIRQ ] 0.0194409  
[ QUBIT ]   2 [ GATE ]  30 [ QS <numpy> ] 0.0070176   [ QS <jax> ] 0.0359228   [ QS <tensorflow> ] 0.0280683   [ QISKIT ] 0.079649    [ CIRQ ] 0.0218699  
[ QUBIT ]   3 [ GATE ]  30 [ QS <numpy> ] 0.0080912   [ QS <jax> ] 0.2334247   [ QS <tensorflow> ] 0.0248401   [ QISKIT ] 0.0735457   [ CIRQ ] 0.0197616  
[ QUBIT ]   4 [ GATE ]  30 [ QS <numpy> ] 0.0081

##### **Fixed Qubit Size 10**

In [0]:
# Fixed Qubit Size 10
cpu = QuantumSimulatorBenchmark(min_qubit_size=10,
                              max_qubit_size=10,
                              min_gate_size=1,
                              max_gate_size=50)
cpu_benchmark_2 = cpu.run_benchmark()

[ CPU ]
processor : 0 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 1 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 2 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 3 Intel(R) Xeon(R) CPU @ 2.30GHz

[ GPU ]
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

[ TPU ]
No TPU found

[ JAX BACKEND ]
[CpuDevice(id=0)]

[ TENSORFLOW BACKEND ]
tensorflow version:  2.1.0
device information: No gpu available

[ QUBIT ]  10 [ GATE ]   1 [ QS <numpy> ] 0.0050216   [ QS <jax> ] 0.0889628   [ QS <tensorflow> ] 0.0157192   [ QISKIT ] 0.0585535   [ CIRQ ] 0.014771   
[ QUBIT ]  10 [ GATE ]   2 [ QS <numpy> ] 0.0039678   [ QS <jax> ] 0.129185    [ QS <tensorflow> ] 0.0179713   [ QISKIT ] 0.0557435   [ CIRQ ] 0.0137053  
[ QUBIT ]  10 [ GATE ]   3 [ QS <numpy> ] 0.0047517   [ QS <jax> ] 0.4257178   [ QS <tensorflow> ] 0.0168633   [ QISKIT ] 0.0547361   [ CIRQ ] 0.0135012  
[ QUBIT ]  10 [ GATE ]   4 [ QS <numpy> ] 0.0059

#### **GPU**

##### **Fixed Gate Size 30**

In [0]:
# Fixed Gate Size 30
gpu = QuantumSimulatorBenchmark(min_qubit_size=23,
                              max_qubit_size=25,
                              min_gate_size=20,
                              max_gate_size=20)
gpu_benchmark_1 = gpu.run_benchmark()

[ CPU ]
processor number:  4
processor : 0 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 1 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 2 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 3 Intel(R) Xeon(R) CPU @ 2.30GHz

memory total :       26753320 kB
memory free :        14492300 kB
memory available :   23128272 kB

[ GPU ]
Wed Feb  5 03:36:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    36W / 250W |  15499MiB / 16280MiB |      0%      Default |
+---------

##### **Fixed Qubit Size 10**

In [0]:
# Fixed Qubit Size 10
gpu = QuantumSimulatorBenchmark(min_qubit_size=10,
                              max_qubit_size=10,
                              min_gate_size=1,
                              max_gate_size=50)
gpu_benchmark_2 = gpu.run_benchmark()

[ CPU ]
processor : 0 Intel(R) Xeon(R) CPU @ 2.20GHz
processor : 1 Intel(R) Xeon(R) CPU @ 2.20GHz
processor : 2 Intel(R) Xeon(R) CPU @ 2.20GHz
processor : 3 Intel(R) Xeon(R) CPU @ 2.20GHz

[ GPU ]
Wed Feb  5 02:33:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    32W / 250W |  15739MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                   

#### **TPU**

##### **Fixed Gate Size 30**

In [0]:
# Fixed Gate Size 30
tpu = QuantumSimulatorBenchmark(min_qubit_size=1,
                              max_qubit_size=23,
                              min_gate_size=30,
                              max_gate_size=30)
tpu_benchmark_1 = tpu.run_benchmark()

[ CPU ]
processor : 0 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 1 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 2 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 3 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 4 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 5 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 6 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 7 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 8 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 9 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 10 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 11 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 12 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 13 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 14 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 15 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 16 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 17 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 18 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 19 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 20 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 21 Intel(R) Xeon(R) CPU 

##### **Fixed Qubit Size 10**

In [0]:
# Fixed Qubit Size 10
tpu = QuantumSimulatorBenchmark(min_qubit_size=10,
                              max_qubit_size=10,
                              min_gate_size=1,
                              max_gate_size=50)
tpu_benchmark_2 = tpu.run_benchmark()

[ CPU ]
processor : 0 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 1 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 2 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 3 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 4 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 5 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 6 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 7 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 8 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 9 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 10 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 11 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 12 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 13 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 14 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 15 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 16 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 17 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 18 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 19 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 20 Intel(R) Xeon(R) CPU @ 2.30GHz
processor : 21 Intel(R) Xeon(R) CPU 

## Save file

In [0]:
# CPU
cpu_benchmark_1.to_csv('cpu_fixed_qubit_1_23_gate_30.csv') 
cpu_benchmark_2.to_csv('cpu_fixed_qubit_10_gate_1_50.csv')
files.download("cpu_fixed_qubit_1_23_gate_30.csv")
files.download("cpu_fixed_qubit_10_gate_1_50.csv") 

In [0]:
cpu_benchmark_1

,qubit,gate,numpy,jax,tensorflow,qiskit,cirq
0,1,30,0.005845,0.032209,0.017044,0.062323,0.019441
1,2,30,0.007018,0.035923,0.028068,0.079649,0.021870
2,3,30,0.008091,0.233425,0.024840,0.073546,0.019762
3,4,30,0.008197,0.295285,0.028659,0.078236,0.022003
4,5,30,0.009122,0.476967,0.030050,0.075967,0.020475
5,6,30,0.010286,0.775410,0.030541,0.083737,0.020544
6,7,30,0.010310,1.975380,0.036427,0.072499,0.020646
7,8,30,0.009873,2.899886,0.031919,0.077392,0.019623
8,9,30,0.011314,2.007261,0.037796,0.073755,0.021666
9,10,30,0.014573,7.207239,0.037883,0.303538,0.021402


In [0]:
cpu_benchmark_2

,qubit,gate,numpy,jax,tensorflow,qiskit,cirq
0,10,1,0.005022,0.088963,0.015719,0.058553,0.014771
1,10,2,0.003968,0.129185,0.017971,0.055743,0.013705
2,10,3,0.004752,0.425718,0.016863,0.054736,0.013501
3,10,4,0.005919,0.542871,0.018360,0.061639,0.014278
4,10,5,0.005644,0.420624,0.019637,0.062434,0.014925
5,10,6,0.006432,0.525048,0.018230,0.060397,0.014738
6,10,7,0.005252,0.884525,0.021660,0.064573,0.015621
7,10,8,0.006666,1.839086,0.021898,0.058559,0.015419
8,10,9,0.006493,0.709385,0.022863,0.061691,0.014646
9,10,10,0.006927,0.860543,0.023756,0.070080,0.015633


In [0]:
# GPU
gpu_benchmark_1.to_csv('gpu_fixed_qubit_1_23_gate_30.csv') 
gpu_benchmark_2.to_csv('gpu_fixed_qubit_10_gate_1_50.csv')
files.download("gpu_fixed_qubit_1_23_gate_30.csv")
files.download("gpu_fixed_qubit_10_gate_1_50.csv") 

In [0]:
gpu_benchmark_1

,qubit,gate,numpy,jax,tensorflow,qiskit,cirq
0,1,30,0.010389,1.339688,0.057678,0.056173,0.020291
1,2,30,0.007092,1.721608,6.433775,0.065978,0.017718
2,3,30,0.006225,1.322002,0.035036,0.068381,0.017477
3,4,30,0.007127,1.264702,0.037926,0.064342,0.017016
4,5,30,0.006716,1.400928,0.043097,0.070097,0.017493
5,6,30,0.007866,2.228712,0.043693,0.066401,0.017553
6,7,30,0.008881,1.503401,0.041178,0.077670,0.017187
7,8,30,0.008004,1.745720,0.038260,0.235651,0.017327
8,9,30,0.009830,1.619210,0.053144,0.081514,0.018220
9,10,30,0.008912,1.584872,0.049518,0.078786,0.019012


In [0]:
gpu_benchmark_2

,qubit,gate,numpy,jax,tensorflow,qiskit,cirq
0,10,1,0.004411,0.047980,0.018283,0.048851,0.010969
1,10,2,0.003699,0.047660,0.021406,0.044044,0.010201
2,10,3,0.003504,0.168280,0.018670,0.050094,0.013455
3,10,4,0.005532,0.342668,0.025699,0.059415,0.013754
4,10,5,0.005030,0.367345,0.028650,0.053388,0.011563
5,10,6,0.004582,0.818132,0.022962,0.051871,0.012515
6,10,7,0.005021,0.425038,0.023860,0.048973,0.013047
7,10,8,0.006509,0.440121,0.024312,0.057434,0.012371
8,10,9,0.005018,0.569309,0.027925,0.050905,0.012155
9,10,10,0.004855,0.449360,0.027032,0.058563,0.013677


In [0]:
# TPU
tpu_benchmark_1.to_csv('tpu_fixed_qubit_1_23_gate_30.csv') 
tpu_benchmark_2.to_csv('tpu_fixed_qubit_10_gate_1_50.csv')
files.download("tpu_fixed_qubit_1_23_gate_30.csv")
files.download("tpu_fixed_qubit_10_gate_1_50.csv")

In [0]:
tpu_benchmark_1

,qubit,gate,numpy,jax,tensorflow,qiskit,cirq
0,1,30,0.013206,0.411508,0.071291,0.069805,0.026842
1,2,30,0.008596,0.901848,0.034019,0.071269,0.020918
2,3,30,0.006965,0.609018,0.024144,0.078991,0.019881
3,4,30,0.009523,0.686429,0.031655,0.067920,0.019695
4,5,30,0.011095,1.119538,0.030005,0.086392,0.020169
5,6,30,0.011070,1.505051,0.037475,0.077103,0.019862
6,7,30,0.008738,2.065831,0.033529,0.084436,0.020240
7,8,30,0.010164,2.379414,0.041537,0.280101,0.020819
8,9,30,0.017742,3.277562,0.036582,0.087170,0.019961
9,10,30,0.012457,1.502304,0.041271,0.088828,0.020841


In [0]:
tpu_benchmark_2

,qubit,gate,numpy,jax,tensorflow,qiskit,cirq
0,10,1,0.006453,0.137862,0.018041,0.084558,0.016819
1,10,2,0.005007,0.219476,0.016918,0.068458,0.013664
2,10,3,0.005373,0.498412,0.018862,0.057959,0.021542
3,10,4,0.005136,0.484710,0.021566,0.067817,0.013016
4,10,5,0.004479,0.909193,0.019640,0.073570,0.016130
5,10,6,0.005551,0.354597,0.027738,0.069177,0.014303
6,10,7,0.005651,1.395780,0.021649,0.069401,0.015954
7,10,8,0.007070,1.117972,0.022304,0.069506,0.020990
8,10,9,0.006153,1.119750,0.022715,0.067653,0.024742
9,10,10,0.006426,1.112062,0.021581,0.065111,0.026347
